<a href="https://colab.research.google.com/github/vasilandrian/vasilandrian-/blob/main/IPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.compose import ColumnTransformer
from keras.callbacks import ModelCheckpoint, EarlyStopping
import requests
import os
from io import BytesIO
import gzip
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
from models_ddos import DDoSModel

epochs = 100
nclass = 12

def loadDataset():
    # Τοποθετήστε τον διαδρομή του dataset εδώ!
    filename = 'https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/pcap_data.csv'

    trainfile = pd.read_csv(filename)
    data = pd.DataFrame(trainfile).to_numpy()
    data = data[data[:, 67] != 'DrDoS_LDAP']
    np.random.shuffle(data)

    label = data[:, 67].astype('str')

    label[label == 'WebDDoS'] = 0
    label[label == 'BENIGN'] = 1
    label[label == 'UDP-lag'] = 2
    label[label == 'DrDoS_NTP'] = 3
    label[label == 'Syn'] = 4
    label[label == 'DrDoS_SSDP'] = 5
    label[label == 'DrDoS_UDP'] = 6
    label[label == 'DrDoS_NetBIOS'] = 7
    label[label == 'DrDoS_MSSQL'] = 8
    label[label == 'DrDoS_SNMP'] = 9
    label[label == 'TFTP'] = 10
    label[label == 'DrDoS_DNS'] = 11

    inx_sel = -1 + np.array([38, 47, 37, 48, 11, 9, 7, 52, 10, 36, 1, 34, 4, 17, 19, 57, 21,
                             18, 22, 24, 32, 50, 23, 55, 51, 5, 3, 39, 40, 43, 58, 12, 25,
                             20, 2, 35, 67, 33, 6, 53])

    data = data[:, inx_sel]
    dmin = data.min(axis=0)
    dmax = data.max(axis=0)
    data = (data - dmin) / (dmax - dmin)

    train_data, test_data, train_label, test_label = \
        train_test_split(data, label, test_size=0.20, stratify=label)

    train_data, val_data, train_label, val_label = \
        train_test_split(train_data, train_label, test_size=0.125, stratify=train_label)

    return train_data.astype('float32'), train_label.astype('int32'), \
           val_data.astype('float32'), val_label.astype('int32'), \
           test_data.astype('float32'), test_label.astype('int32')

train_data, train_labelp, val_data, val_labelp, test_data, test_labelp = loadDataset()

train_label = to_categorical(train_labelp, nclass)
val_label = to_categorical(val_labelp, nclass)
test_label = to_categorical(test_labelp, nclass)

print('train_data.shape=', train_data.shape)
print('test_data.shape=',  test_data.shape)
print('val_data.shape=',  val_data.shape)

inshape = train_data.shape[1]

class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(train_labelp),
                                                  y=train_labelp)

class_weights = {i: class_weights[i] for i in range(len(class_weights))}

earlyStopping = EarlyStopping(monitor='val_loss',
                              patience=30,
                              verbose=0,
                              mode='min')

modelCheckPoint = ModelCheckpoint('./savemodels/model5class.weights.{epoch:03d}-{val_acc:.4f}.hdf5',
                                  save_best_only=True,
                                  monitor='val_acc',
                                  mode='max')

model = DDoSModel.model_conv1D(lr=1e-4, N=64, inshape=inshape)
model.summary()

history = model.fit(train_data,
                    train_label,
                    shuffle=True,
                    epochs=epochs,
                    batch_size=256,
                    validation_data=(val_data, val_label),
                    callbacks=[modelCheckPoint],
                    class_weight=class_weights,
                    workers=3)

str_models = os.listdir('./savemodels')
str_models = np.sort(str_models)
best_model = str_models[str_models.size-1]
print('best_model=', best_model)
model.load_weights('./savemodels/'+best_model)

pred = model.predict(test_data)
pred_y = pred.argmax(axis=-1)

cm = confusion_matrix(test_labelp.astype('int32'), pred_y)
print(cm)

label = np.array(["WebDDoS", "BENIGN", "UDP-lag", "DrDoS_NTP", "Syn",
                  "DrDoS_SSDP", "DrDoS_UDP", "DrDoS_NetBIOS", "DrDoS_MSSQL",
                  "DrDoS_SNMP", "TFTP", "DrDoS_DNS"])

cmo = ConfusionMatrixDisplay(cm, display_labels=label)
fig, ax = plt.subplots(figsize=(12, 12))
cmo.plot(ax=ax, xticks_rotation=45)

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

np.save('historydata.npy', [acc, val_acc, loss, val_loss])
[acc, val_acc, loss, val_loss] = np.load('historydata.npy')

plt.figure()
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r.', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r.', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
